Vamos a meter el nivel de torneo

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from ranking_scores import puntos_ranking

In [2]:
def calculate_head_to_head(dataset):
    # Convertir la columna 'tourney_date' a formato de fecha
    dataset['tourney_date'] = pd.to_datetime(dataset['tourney_date'], format='%Y%m%d')

    # Ordenar el dataset por fecha
    dataset = dataset.sort_values('tourney_date')

    # Inicializar una nueva columna para el historial de enfrentamientos directos
    dataset['head_to_head'] = 0.5
    '''dataset['num_matches'] = 0'''
    # Iterar sobre las filas del dataset
    for index, row in dataset.iterrows():
        # Obtener los partidos anteriores entre los dos jugadores
        previous_matches = dataset[((dataset['winner_id'] == row['winner_id']) & (dataset['loser_id'] == row['loser_id']) | (dataset['winner_id'] == row['loser_id']) & (dataset['loser_id'] == row['winner_id'])) & (dataset['tourney_date'] < row['tourney_date'])]

        if not previous_matches.empty:
            # Calcular el porcentaje de veces que el 'winner_id' ha ganado
            wins = len(previous_matches[previous_matches['winner_id'] == row['winner_id']])
            total = len(previous_matches)
            dataset.at[index, 'head_to_head'] = wins / total
            '''# Calcular el número de partidos jugados
            dataset.at[index, 'num_matches'] = total'''

    return dataset

In [3]:
# Cargar los datos de los años
data_2021 = pd.read_csv('atp_matches_2021.csv')
data_2022 = pd.read_csv('atp_matches_2022.csv')
data_2023 = pd.read_csv('atp_matches_2023.csv')

# Agregar una columna 'source' a cada DataFrame
data_2021['source'] = 'atp_matches_2021.csv'
data_2022['source'] = 'atp_matches_2022.csv'
data_2023['source'] = 'atp_matches_2023.csv'

# Combinar los datos en un solo DataFrame
data_all = pd.concat([data_2021, data_2022, data_2023])

# Aplicar la función calculate_head_to_head
data_all = calculate_head_to_head(data_all)

# Filtrar los datos para obtener solo los partidos de 2023 del archivo 'atp_matches_2023.csv'
dataset = data_all[data_all['source'] == 'atp_matches_2023.csv']

In [4]:
tenistas = pd.concat([dataset['winner_id'], dataset['loser_id']])
tenistas = set(tenistas)
num_tenistas = len(tenistas)

In [5]:
dataset_r = pd.read_csv('atp_rankings_20s.csv')

### Realizamos el tratamiento de los NaN's

In [6]:
# Encontramos el valor más común entre 'Grass', 'Hard', y 'Clay'
most_common = dataset[dataset['surface'].isin(['Grass', 'Hard', 'Clay'])]['surface'].mode()[0]

# Reemplazamos los valores que no son 'Grass', 'Hard', o 'Clay' (los NaN's) con el valor más común
dataset.loc[~dataset['surface'].isin(['Grass', 'Hard', 'Clay']), 'surface'] = most_common

In [7]:

print(set(dataset['tourney_level']))

{'M', 'F', 'G', 'D', 'A'}


In [8]:
pk = puntos_ranking(dataset, dataset_r, 2023)

c:\Users\Guille\OneDrive\Escritorio\Deepty Projects\ranking_scores.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['tourney_date'] = pd.to_datetime(dataset['tourney_date'], format='%Y%m%d')
c:\Users\Guille\OneDrive\Escritorio\Deepty Projects\ranking_scores.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['year_month'] = dataset['tourney_date'].dt.to_period('M')


In [9]:
dataset.loc[:, 'winner_rank_points'], dataset.loc[:, 'loser_rank_points'] = pk['winner_points'], pk['loser_points']

In [10]:
dataset.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,source,head_to_head,year_month
69,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,279,207733,NaN,NaN,...,8.0,7.0,12.0,42.0,990.0,83.0,630.0,atp_matches_2023.csv,0.5,2023-01
70,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,278,111575,8.0,NaN,...,9.0,8.0,12.0,20.0,1990.0,57.0,813.0,atp_matches_2023.csv,0.5,2023-01
71,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,277,206173,6.0,NaN,...,8.0,1.0,4.0,15.0,2410.0,582.0,52.0,atp_matches_2023.csv,1.0,2023-01
72,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,276,106423,NaN,WC,...,12.0,5.0,6.0,93.0,592.0,34.0,1163.0,atp_matches_2023.csv,0.5,2023-01
76,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,272,111456,NaN,NaN,...,5.0,0.0,2.0,63.0,775.0,67.0,741.0,atp_matches_2023.csv,0.5,2023-01


In [11]:
'''Indice 7: ganador del partido.
Indice 15: perdedor del partido.
Indice -1: puntos en el ranking del jugador ganador
Indice -3: puntos en el ranking del jugador perdedor
Indice 2: tipo de campo del partido (grass, hard o clay)'''
'''X = dataset.iloc[:, [7, 15, -1, -3, 2]].values
y = dataset.iloc[:, 7].values'''

'X = dataset.iloc[:, [7, 15, -1, -3, 2]].values\ny = dataset.iloc[:, 7].values'

In [12]:
columnas_X = ['winner_id', 'loser_id', 'winner_rank_points', 'loser_rank_points', 'head_to_head', 'tourney_level', 'surface']
columna_y = 'winner_id'
X = dataset[columnas_X].values
y = dataset[columna_y].values


In [13]:
X[:, 0] = X[:, 0].astype(str)
X[:, 1] = X[:, 1].astype(str)
y = y.astype(str)

In [14]:
# Crear el codificador
encoder = OneHotEncoder(sparse_output=False)

# Ajustar el codificador con los datos de ambos jugadores y 'y'
encoder.fit(X[:, [0, 1]].reshape(-1, 1))

# Transformar las columnas de los jugadores
player1_onehot = encoder.transform(X[:, 0].reshape(-1, 1))
player2_onehot = encoder.transform(X[:, 1].reshape(-1, 1))

# Transformar el vector objetivo 'y' con el mismo codificador 'X'
y = encoder.transform(y.reshape(-1, 1))

# Concatenar las columnas one-hot con el resto de tus datos
X = np.concatenate([player1_onehot, player2_onehot, X[:, 2:]], axis=1)


In [15]:
# Matriz de booleanos aleatorios del mismo tamaño que tus datos
swap = np.random.rand(len(X)) > 0.5

# Intercambiamos las posiciones de los jugadores en 'X' donde 'swap' es True
X[swap, :num_tenistas], X[swap, num_tenistas:num_tenistas*2] = X[swap, num_tenistas:num_tenistas*2], X[swap, :num_tenistas].copy()

# Intercambiamos las posiciones de los puntos en el ranking donde 'swap' es True
X[swap, num_tenistas*2], X[swap, num_tenistas*2+1] = X[swap, num_tenistas*2+1], X[swap, num_tenistas*2].copy()

mask = (swap) & (X[:, -2] != -1)
X[mask, -3] = 1 - X[mask, -3]

In [16]:
from sklearn.preprocessing import StandardScaler

# Crear el escalador
scaler = StandardScaler()

X[:, [-5, -4]] = scaler.fit_transform(X[:, [-5, -4]])

In [17]:
# Esto es para el tipo de campo
# Crear el codificador
encoder2 = OneHotEncoder(sparse_output=False)

# Ajustar y transformar la columna del tipo de campo
field_type_onehot = encoder2.fit_transform(X[:, num_tenistas*2+4].reshape(-1, 1))

# Concatenar la columna one-hot con el resto de los datos
X = np.concatenate([X[:, :num_tenistas*2+4], field_type_onehot], axis=1)

In [18]:
# Esto es para el nivel del torneo
# Crear el codificador
encoder3 = OneHotEncoder(sparse_output=False)

# Ajustar y transformar la columna del tipo de campo
tourney_level_onehot = encoder2.fit_transform(X[:, num_tenistas*2+3].reshape(-1, 1))

# Concatenar la columna one-hot con el resto de los datos
X = np.concatenate([X[:, :num_tenistas*2+3], tourney_level_onehot, X[:, num_tenistas*2+4:]], axis=1)

In [19]:
# Primero, obtenemos las 440 primeras columnas de X
M = X[:, :440]

# Luego, agregamos la columna 880
M = np.hstack((M, X[:, 880].reshape(-1, 1)))

# Luego, agregamos las columnas entre las 440 y la 880
M = np.hstack((M, X[:, 440:880]))

# Luego, agregamos la columna 882
M = np.hstack((M, X[:, 881].reshape(-1, 1)))

# Finalmente, agregamos las columnas del final
M = np.hstack((M, X[:, 882:]))

In [20]:
X = M

In [21]:
# Luego, dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [22]:
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
# Vamos a inferir la media poblacional a partir de una muestra aleatoria simple con n = 300
muestras = [0 for _ in range(300)]
for i in range(300):
    
    X_train = X_train.astype('float64')
    y_train = y_train.astype('float64')
    # Definimos el modelo
    model = Sequential()

    # Añadir la capa de entrada y la primera capa oculta
    model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))
    # model.add(BatchNormalization())
    # model.add(Dropout(0.2))
    # Añadir la segunda capa oculta
    # model.add(Dense(units=32, activation='tanh'))

    # Añadir la capa de salida
    model.add(Dense(units=y.shape[1], activation='softmax'))

    # Compilar el modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=15, batch_size=32)

    X_test = X_test.astype('float64')
    y_test = y_test.astype('float64')

    # Hacemos la predicción
    prediccion_test = model.predict(X_test)

    # Obtenemos los índices donde hay un 1 en las primeras 440 columnas y en las siguientes 440 columnas
    indices_1 = np.where(X_test[:, :num_tenistas] == 1)[1]
    indices_2 = np.where(X_test[:, num_tenistas:num_tenistas*2] == 1)[1]

    # Obtenemos las predicciones para los índices obtenidos
    predicciones_1 = prediccion_test[np.arange(len(prediccion_test)), indices_1]
    predicciones_2 = prediccion_test[np.arange(len(prediccion_test)), indices_2]

    # Encontramos el índice del máximo para cada fila
    max_indices = np.where(predicciones_1 > predicciones_2, indices_1, indices_2)

    # Convertimos max_indices en un array 2D
    max_indices_2D = np.eye(encoder.categories_[0].shape[0])[max_indices]

    # Usamos el codificador para transformar los índices a sus etiquetas originales
    etiquetas_predichas = encoder.inverse_transform(max_indices_2D)
    etiqueta_y_test = encoder.inverse_transform(y_test)
    # Veamos cuántos partidos ha acertado para el test
    aciertos = 0
    for j in range(len(etiquetas_predichas)):
        if etiquetas_predichas[j] == etiqueta_y_test[j]:
            aciertos += 1
    precision = aciertos / len(etiquetas_predichas)*100
    muestras[i] = precision
    print(i, precision)

Epoch 1/15
75/75 [==============================] - 1s 2ms/step - loss: 5.6947 - accuracy: 0.0611
Epoch 2/15
75/75 [==============================] - 0s 3ms/step - loss: 4.5725 - accuracy: 0.1034
Epoch 3/15
75/75 [==============================] - 0s 2ms/step - loss: 4.1124 - accuracy: 0.2902
Epoch 4/15
75/75 [==============================] - 0s 3ms/step - loss: 3.7055 - accuracy: 0.4359
Epoch 5/15
75/75 [==============================] - 0s 3ms/step - loss: 3.2724 - accuracy: 0.5339
Epoch 6/15
75/75 [==============================] - 0s 2ms/step - loss: 2.8265 - accuracy: 0.5955
Epoch 7/15
75/75 [==============================] - 0s 3ms/step - loss: 2.4038 - accuracy: 0.6512
Epoch 8/15
75/75 [==============================] - 0s 2ms/step - loss: 2.0150 - accuracy: 0.6893
Epoch 9/15
75/75 [==============================] - 0s 2ms/step - loss: 1.6799 - accuracy: 0.7228
Epoch 10/15
75/75 [==============================] - 0s 3ms/step - loss: 1.3969 - accuracy: 0.7567
Epoch 11/15
75/75 [

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

# Calcular la media y la desviación estándar de las muestras
mu, std = np.mean(muestras), np.std(muestras)

# Generar la distribución gaussiana
gaussian = norm(loc=mu, scale=std)

# Crear un array de valores x para la gráfica
x = np.linspace(min(muestras), max(muestras), 100)

# Crear la gráfica
plt.figure(figsize=(10, 6))
plt.hist(muestras, bins=20, density=True, alpha=0.6, color='g')
plt.plot(x, gaussian.pdf(x), 'r', linewidth=2)
plt.title('Distribución de las predicciones y la gaussiana')
plt.xlabel('Precisión')
plt.ylabel('Frecuencia')
plt.title('Distribución muestral de la precisión con 300 muestras')
plt.show()
print(mu, std, np.median(muestras))

In [ ]:
constante = 1
W = np.full((X_train.shape[1], 128), constante)  
# Definimos el modelo
model = Sequential()

# Añadir la capa de entrada y la primera capa oculta
model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# Añadir la segunda capa oculta
# model.add(Dense(units=32, activation='tanh'))

# Añadir la capa de salida
model.add(Dense(units=y.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=15, batch_size=32)

Epoch 1/15
75/75 [==============================] - 1s 2ms/step - loss: 5.6860 - accuracy: 0.0452
Epoch 2/15
75/75 [==============================] - 0s 2ms/step - loss: 4.5601 - accuracy: 0.1156
Epoch 3/15
75/75 [==============================] - 0s 2ms/step - loss: 4.1021 - accuracy: 0.3145
Epoch 4/15
75/75 [==============================] - 0s 2ms/step - loss: 3.6954 - accuracy: 0.4657
Epoch 5/15
75/75 [==============================] - 0s 2ms/step - loss: 3.2642 - accuracy: 0.5494
Epoch 6/15
75/75 [==============================] - 0s 2ms/step - loss: 2.8172 - accuracy: 0.6223
Epoch 7/15
75/75 [==============================] - 0s 2ms/step - loss: 2.3890 - accuracy: 0.6524
Epoch 8/15
75/75 [==============================] - 0s 2ms/step - loss: 2.0053 - accuracy: 0.6884
Epoch 9/15
75/75 [==============================] - 0s 3ms/step - loss: 1.6666 - accuracy: 0.7219
Epoch 10/15
75/75 [==============================] - 0s 3ms/step - loss: 1.3849 - accuracy: 0.7487
Epoch 11/15
75/75 [

In [ ]:
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [ ]:
# Hacemos la predicción
prediccion_test = model.predict(X_test)

# Obtenemos los índices donde hay un 1 en las primeras 440 columnas y en las siguientes 440 columnas
indices_1 = np.where(X_test[:, :440] == 1)[1]
indices_2 = np.where(X_test[:, 441:881] == 1)[1]

# Obtenemos las predicciones para los índices obtenidos
predicciones_1 = prediccion_test[np.arange(len(prediccion_test)), indices_1]
predicciones_2 = prediccion_test[np.arange(len(prediccion_test)), indices_2]

# Encontramos el índice del máximo para cada fila
max_indices = np.where(predicciones_1 > predicciones_2, indices_1, indices_2)

# Convertimos max_indices en un array 2D
max_indices_2D = np.eye(encoder.categories_[0].shape[0])[max_indices]

# Usamos el codificador para transformar los índices a sus etiquetas originales
etiquetas_predichas = encoder.inverse_transform(max_indices_2D)

print(etiquetas_predichas)

19/19 [==============================] - 0s 1ms/step
[['105777']
 ['106043']
 ['136440']
 ['208103']
 ['200279']
 ['134868']
 ['105870']
 ['210097']
 ['200303']
 ['208286']
 ['106421']
 ['208286']
 ['111200']
 ['122330']
 ['126205']
 ['207678']
 ['100644']
 ['126094']
 ['104527']
 ['126207']
 ['208103']
 ['210506']
 ['202104']
 ['105732']
 ['200282']
 ['106218']
 ['111456']
 ['126203']
 ['106121']
 ['111797']
 ['128034']
 ['206909']
 ['126239']
 ['126127']
 ['134770']
 ['126203']
 ['122298']
 ['124014']
 ['106148']
 ['104545']
 ['144869']
 ['100644']
 ['106233']
 ['104527']
 ['126094']
 ['106432']
 ['200221']
 ['207686']
 ['126205']
 ['105550']
 ['106043']
 ['100644']
 ['106234']
 ['208293']
 ['144707']
 ['106218']
 ['126774']
 ['126205']
 ['134770']
 ['111513']
 ['202103']
 ['124079']
 ['200267']
 ['111513']
 ['104755']
 ['111797']
 ['104755']
 ['111456']
 ['106331']
 ['207733']
 ['200059']
 ['207989']
 ['111153']
 ['106421']
 ['206173']
 ['210178']
 ['208029']
 ['126094']
 ['106331']

In [ ]:
# Usamos el codificador para transformar las etiquetas codificadas a sus etiquetas originales
etiqueta_y_test = encoder.inverse_transform(y_test)

print(etiqueta_y_test)

[['105777']
 ['106043']
 ['136440']
 ['208103']
 ['105656']
 ['134868']
 ['202103']
 ['210097']
 ['200303']
 ['207518']
 ['105777']
 ['144719']
 ['209113']
 ['122330']
 ['105430']
 ['207678']
 ['111442']
 ['126094']
 ['104527']
 ['126207']
 ['208103']
 ['210506']
 ['105948']
 ['105732']
 ['200282']
 ['106426']
 ['111456']
 ['105777']
 ['106121']
 ['122330']
 ['128034']
 ['209950']
 ['126239']
 ['126127']
 ['106121']
 ['200282']
 ['122298']
 ['105359']
 ['106148']
 ['200059']
 ['200624']
 ['104918']
 ['106233']
 ['105777']
 ['126094']
 ['202195']
 ['200221']
 ['207686']
 ['126205']
 ['106198']
 ['106043']
 ['100644']
 ['106234']
 ['208293']
 ['123755']
 ['126239']
 ['126774']
 ['206909']
 ['105526']
 ['111513']
 ['200000']
 ['124079']
 ['111805']
 ['111513']
 ['104755']
 ['111797']
 ['104755']
 ['208029']
 ['106331']
 ['111575']
 ['144684']
 ['207989']
 ['106214']
 ['106421']
 ['206173']
 ['126762']
 ['208029']
 ['126094']
 ['106331']
 ['132283']
 ['111456']
 ['104925']
 ['208859']
 ['2

In [ ]:
# Veamos cuántos partidos ha acertado para el test
aciertos = sum(etiquetas_predichas == etiqueta_y_test)
precision = aciertos / len(etiquetas_predichas)*100
print(f"Precisión del test: {np.round(precision[0], 2)}%")

Precisión del test: 65.72%
